In [130]:
sc
import pyspark.sql.functions as sf

In [132]:
dfTweets = sqlContext.read.load('stream_twt_april28.jsonl', format='json')
# dfTweets.printSchema()

In [133]:
retweet_status = dfTweets.select('place.bounding_box', sf.explode('quoted_status.entities.hashtags.text').alias('text'),'place.full_name', 'timestamp_ms')
retweet_status.registerTempTable('toplocations')
# retweet_status.printSchema()

In [134]:

occurrence = sqlContext.sql('select bounding_box, full_name, text ,timestamp_ms as occurrence from toplocations').dropna()
# occurrence = occurrence.filter(sf.size('text') > 0)
# toploc = occurrence.orderBy('occurrence', ascending=False) #order by descending order based on occurrence
occurrence.show()
#occurrence.count()

+--------------------+-------------------+--------------------+-------------+
|        bounding_box|          full_name|                text|   occurrence|
+--------------------+-------------------+--------------------+-------------+
|[WrappedArray(Wra...|          Wayne, NJ|             TheList|1493392012864|
|[WrappedArray(Wra...|     Sacramento, CA|               photo|1493392016065|
|[WrappedArray(Wra...|     Sacramento, CA|                parc|1493392016065|
|[WrappedArray(Wra...|        Houston, TX|MakeTodayBetterIn...|1493392017103|
|[WrappedArray(Wra...| Palms, Los Angeles|        First100days|1493392019758|
|[WrappedArray(Wra...| Kitchener, Ontario|           NHLBruins|1493392019951|
|[WrappedArray(Wra...|  San Francisco, CA|        FyreFestival|1493392022821|
|[WrappedArray(Wra...|Shorewood Hills, WI|            DJKHALED|1493392023145|
|[WrappedArray(Wra...|Shorewood Hills, WI|            IMTHEONE|1493392023145|
|[WrappedArray(Wra...|         Hallie, WI|          AWKOHAWNOH|1

In [135]:
# This command is what we use in the cluster.
occurrence.rdd.saveAsTextFile('hashtagmap2')

## After running the code above on cluster

In [136]:
toplocations = sc.textFile("hashtagmap2")

In [137]:
toplocations.take(1)

[u"Row(bounding_box=Row(coordinates=[[[-74.289406, 40.88257], [-74.289406, 41.014136], [-74.195413, 41.014136], [-74.195413, 40.88257]]], type=u'Polygon'), full_name=u'Wayne, NJ', text=u'TheList', occurrence=u'1493392012864')"]

In [138]:
remove = toplocations.map(lambda x: x.replace("Row(bounding_box=Row(coordinates=[[[", ""))
remove = remove.map(lambda x: x.replace("]], type=u'Polygon'), full_name=u'", ","))
remove = remove.map(lambda x: x.replace(", occurrence=u", ";"))
remove = remove.map(lambda x: x.replace(", text=u", ";"))
final = remove.map(lambda x: x.replace(")", "").replace("],", ';').replace('[', '').replace(']', '').replace("'", ''))

final.take(5)

[u'-74.289406, 40.88257; -74.289406, 41.014136; -74.195413, 41.014136; -74.195413, 40.88257;Wayne, NJ;TheList;1493392012864',
 u'-121.576613, 38.43792; -121.576613, 38.685524; -121.362715, 38.685524; -121.362715, 38.43792;Sacramento, CA;photo;1493392016065',
 u'-121.576613, 38.43792; -121.576613, 38.685524; -121.362715, 38.685524; -121.362715, 38.43792;Sacramento, CA;parc;1493392016065',
 u'-95.823268, 29.522325; -95.823268, 30.154665; -95.069705, 30.154665; -95.069705, 29.522325;Houston, TX;MakeTodayBetterIn4Words;1493392017103',
 u'-118.431436, 34.011385; -118.431436, 34.037107; -118.387635, 34.037107; -118.387635, 34.011385;Palms, Los Angeles;First100days;1493392019758']

In [139]:
def mapper(k1v1s): 
    for k1v1 in k1v1s:
        k = k1v1.split(';')
        x_co = float(k[0].split(',')[0]) + float(k[1].split(',')[0])+ float(k[2].split(',')[0])+float(k[3].split(',')[0])
        x_co = x_co/4
        y_co = float(k[0].split(',')[1]) + float(k[1].split(',')[1])+ float(k[2].split(',')[1])+float(k[3].split(',')[1])
        y_co = y_co/4
        yield (round(y_co,6), round(x_co, 6), k[4], str(k[5]).lower(),int(k[6]))


coordinates = final.mapPartitions(mapper)
coordinates.take(1)
print (type(coordinates))

<class 'pyspark.rdd.PipelinedRDD'>


In [142]:
def mapper_hashtags(k1v1s):
    for k1v1 in k1v1s:
        yield (k1v1[2],(k1v1[3]+' '))
        
def mapper2((key, value)):
    for word in value.split():
        if len(word)>0:
            yield ((key,word),1) 
            
def mapper3(k1v1s):
    for k,v in k1v1s:
        yield (k,v)
        
def reducer3((key, value)):
    return (key , value)
    
        
#     return (k1, summ)

# coordinates.mapPartitions(mapper_hashtags).groupByKey().collect()

# coordinates.mapPartitions(mapper_hashtags).mapValues(mapper2).take(5) #reduce(reducer_hash).take(10)
coo = coordinates.mapPartitions(mapper_hashtags).reduceByKey(lambda x,y: (x+y))
# coo.take(5)

f = coo.flatMap(mapper2).groupByKey().mapValues(lambda values: sum(values))
# f.take(10)
# f.mapValues(lambda x : x > 2).take(10)
f = f.toLocalIterator()

# f = f.toLocalIterator()
# print next(coord)[0]

In [143]:
loclist = [] 

for i in f:
    if i[1] >= 5:
        loclist.append([i[0][0],i[0][1],i[1]])

# print loclist

In [144]:
print loclist


[[u'Washington, DC', 'dcfamily', 7], [u'Manhattan, NY', 'fyrefestival', 16], [u'Montgomery, AL', 'myasu', 21], [u'Florida, USA', 'maga', 6], [u'Washington, USA', 'featuring', 15], [u'Florida, USA', 'jaguars', 5], [u'Waterbury, CT', 'russia', 22], [u'Washington, USA', 'hot', 5], [u'Washington, USA', 'hitsingle', 15], [u'Waterbury, CT', 'foxnews', 22], [u'Houston, TX', 'nfldraft', 16], [u'Washington, USA', 'iknowyouroutthere', 5], [u'Surrey, British Columbia', 'bcpoli', 5], [u'Georgia, USA', 'nfldraft', 14], [u'Huntsville, TX', 'shsuadpiderbydays2k17', 5], [u'Rockville, MD', 'grime', 5], [u'Washington, USA', 'kickassmusic', 5], [u'Knoxville, TN', 'nfldraft', 10], [u'Fort Worth, TX', 'nfldraft', 5], [u'Rockville, MD', 'nyhiphop', 5], [u'Los Angeles, CA', 'fyrefestival', 28], [u'Hemet, CA', 'retweet', 5], [u'Denton, TX', 'hepoolshenanigans', 5], [u'Seattle, WA', 'fyrefestival', 14], [u'Washington, USA', 'follow', 15], [u'Washington, USA', 'discover', 15], [u'Riverside, CA', 'springsplashuc

In [145]:
import csv

def writeRecords(records):
    with open('hashtagmap2.csv', 'w') as csvfile:
        field = ['location_name', 'hash_tag', 'ht_count']
        writer = csv.DictWriter(csvfile, fieldnames=field)

        writer.writeheader()
        for record in records:
            writer.writerow({'location_name': record[0], 'hash_tag': record[1], 'ht_count': record[2]})

In [146]:
# coo = coordinates.toLocalIterator()
writeRecords(loclist)

In [ ]:
coo.registerTempTable('toplocations')

In [ ]:
result = timeline.join(timeline2, timeline.date == timeline2.date_)